<a href="https://colab.research.google.com/github/mmdreza-ghorbani/Thesis_codes/blob/main/Stack%20rasters%20Data%20and_Create%20Asmethic%20Shape%20Atrribute.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import modules

In [ ]:
!pip install geopandas
!pip install rasterio



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 56.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 85.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 61.6 MB/s eta 0:00:00


In [ ]:

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio
import rasterio.mask
from rasterio.plot import show
from rasterio.transform import Affine
from scipy.spatial import Voronoi, voronoi_plot_2d
from shapely.geometry import box
from shapely.geometry import Polygon, Point
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor

get access from user

In [ ]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [ ]:
def prepareShape(srcpath,distpath,newfileName):
    # --------------------------------------------  change shape
  # Open the raster that you want to modify
  src_dataset = gdal.Open(srcpath, gdal.GA_ReadOnly)

  # Open the target raster that has the desired shape
  target_dataset = gdal.Open(distpath, gdal.GA_ReadOnly)

  # Get the target extent and resolution
  target_transform = target_dataset.GetGeoTransform()
  target_width = target_dataset.RasterXSize
  target_height = target_dataset.RasterYSize

  # Create a new raster file with the desired shape
  driver = gdal.GetDriverByName('GTiff')
  output_dataset = driver.Create(
    newfileName,
    target_width,
    target_height,
    src_dataset.RasterCount,
    src_dataset.GetRasterBand(1).DataType
  )

  # Set the projection and transformation for the output raster
  output_dataset.SetProjection(target_dataset.GetProjection())
  output_dataset.SetGeoTransform(target_transform)

  # Reproject and resample the source raster to match the target raster
  gdal.ReprojectImage(
    src_dataset,
    output_dataset,
    src_dataset.GetProjection(),
    target_dataset.GetProjection(),
    gdal.GRA_NearestNeighbour
  )

  #  Close the datasets
  src_dataset = None
  target_dataset = None
  output_dataset = None

In [ ]:
new_cell_size = 20
def prepareRasterForStacked(srcpath,distpath,newfileName):
   #--------------------RESAMPLE--------------------
  with rasterio.open(srcpath) as src:

    # Calculate the new dimensions of the output raster based on the desired cell size
    width = int(src.width * src.res[0] / new_cell_size)
    height = int(src.height * src.res[1] / new_cell_size)

    # Calculate the new geotransform of the output raster
    transform = rasterio.Affine(src.transform.a / (src.res[0] / new_cell_size), src.transform.b, src.transform.c,
                                src.transform.d, src.transform.e / (src.res[1] / new_cell_size), src.transform.f)
    # Create the output raster dataset with the new dimensions and geotransform
    meta = src.meta.copy()
    meta.update({
        'width': width,
        'height': height,
        'transform': transform,
    })
    with rasterio.open(srcpath, 'w', **meta) as dst:
      for i in range(1, src.count + 1):
            raster = src.read(i, out_shape=(src.count, height, width))
            dst.write(raster, i)
  # --------------------------------------------  change shape
  prepareShape(srcpath,distpath,newfileName)

In [ ]:
TifExportUrl = '/content/drive/My Drive/Tif_Export/'
ShapefilesURL='/content/drive/My Drive/ShapeFiles/'
mainURL = '/content/drive/My Drive/'
AirPolutionURL='/content/drive/My Drive/AIRNOW/'

from osgeo import gdal
climatologyDF = gpd.read_file(ShapefilesURL+"climatologyMean.shp")
# Open raster
#raster_pk = rasterio.open( TifExportUrl + variableName + "_Map.tif")
from osgeo import gdal
import numpy as np
file_list = []
for i in range(len(climatologyDF.columns)):
     variableName = climatologyDF.columns[i]
     if(variableName == 'geometry' or variableName  == 'station_id'):
       continue
     file_list.append(TifExportUrl + variableName + "_Map_Extend.tif")
     prepareRasterForStacked(TifExportUrl + variableName + "_Map.tif",TifExportUrl + 'Tehran_NDVIimage_sentinel20MUTM.tif', TifExportUrl + variableName + "_Map_Extend.tif")

prepareRasterForStacked(AirPolutionURL + "airpolution_pk_kriging.tif",mainURL + 'Tehran_NDVIimage_sentinel20MUTM.tif', mainURL + 'newAirpolution.tif')
#prepareShape(mainURL + 'LandUse_main_20m.tif',mainURL + 'Tehran_NDVIimage_sentinel20MUTM.tif',mainURL + 'newLandUse.tif')
prepareRasterForStacked(mainURL + 'LanuseGrayScale.tif',mainURL + 'Tehran_NDVIimage_sentinel20MUTM.tif',mainURL + 'newLandUse.tif')
prepareRasterForStacked(mainURL + 'ALOS_Tehran_SlopeUTM10.tif',mainURL + 'Tehran_NDVIimage_sentinel20MUTM.tif',mainURL + 'newSlop.tif')
prepareRasterForStacked(mainURL + 'ALOS_Tehran_ElevationUTM10.tif',mainURL + 'Tehran_NDVIimage_sentinel20MUTM.tif',mainURL + 'newElavation.tif')
prepareRasterForStacked(mainURL + 'DistanceToRoad.tif',mainURL + 'Tehran_NDVIimage_sentinel20MUTM.tif',mainURL + 'newDistanceToRoad.tif')
file_list.append(mainURL + 'newSlop.tif')
file_list.append(mainURL + 'newElavation.tif')
file_list.append(mainURL + 'newDistanceToRoad.tif')
file_list.append( mainURL + 'Tehran_NDVIimage_sentinel20MUTM.tif')
file_list.append(mainURL + 'newAirpolution.tif')
file_list.append(mainURL + 'newLandUse.tif')


In [ ]:
raster_pk = rasterio.open(file_list[18])
display(raster_pk.crs)
print('shape: ')
print(raster_pk.shape)
out_image_pk, out_transform_pk = rasterio.mask.mask(raster_pk, climatologyDF.geometry.values, crop = True)
print('max: ')
print( out_image_pk.max())
print('min: ')
print( out_image_pk.min())
pixelSizeX, pixelSizeY  = raster_pk.res
print('pixelSizeX: ')
print(pixelSizeX)
print('pixelSizeY: ')
print(pixelSizeY)

CRS.from_epsg(32639)

shape: 
(1461, 2351)
max: 
7.321781
min: 
0.0
pixelSizeX: 
20.0
pixelSizeY: 
20.0


In [ ]:
raster_pk = rasterio.open( mainURL + 'Tehran_NDVIimage_sentinel20MUTM.tif')
display(raster_pk.crs)
print('shape: ')
print(raster_pk.shape)
out_image_pk, out_transform_pk = rasterio.mask.mask(raster_pk, climatologyDF.geometry.values, crop = True)
print('max: ')
print( out_image_pk.max())
print('min: ')
print( out_image_pk.min())
pixelSizeX, pixelSizeY  = raster_pk.res
print('pixelSizeX: ')
print(pixelSizeX)
print('pixelSizeY: ')
print(pixelSizeY)

CRS.from_epsg(32639)

shape: 
(1461, 2351)
max: 
0.07056378
min: 
0.0
pixelSizeX: 
20.0
pixelSizeY: 
20.0


In [ ]:
display(file_list)

['/content/drive/My Drive/Tif_Export/h_Map_Extend.tif',
 '/content/drive/My Drive/Tif_Export/vv_Map_Extend.tif',
 '/content/drive/My Drive/Tif_Export/n_Map_Extend.tif',
 '/content/drive/My Drive/Tif_Export/dd_Map_Extend.tif',
 '/content/drive/My Drive/Tif_Export/ff_Map_Extend.tif',
 '/content/drive/My Drive/Tif_Export/t_Map_Extend.tif',
 '/content/drive/My Drive/Tif_Export/td_Map_Extend.tif',
 '/content/drive/My Drive/Tif_Export/p0_Map_Extend.tif',
 '/content/drive/My Drive/Tif_Export/rrr_Map_Extend.tif',
 '/content/drive/My Drive/Tif_Export/u_Map_Extend.tif',
 '/content/drive/My Drive/Tif_Export/tmin_Map_Extend.tif',
 '/content/drive/My Drive/Tif_Export/tmax_Map_Extend.tif',
 '/content/drive/My Drive/Tif_Export/radglo24_Map_Extend.tif',
 '/content/drive/My Drive/Tif_Export/twet_Map_Extend.tif',
 '/content/drive/My Drive/Tif_Export/ff_max_Map_Extend.tif',
 '/content/drive/My Drive/Tif_Export/dd_max_Map_Extend.tif',
 '/content/drive/My Drive/Tif_Export/ff_gust_Map_Extend.tif',
 '/conten

# Stack Data

read path of tiff

create coordinate layer

In [ ]:
# Assuming your raster
raster_shape = raster_pk.shape
# Create two coordinate layers
pixel_x_layer = np.zeros((raster_shape[:]), dtype=np.int32)  # Create pixel_x layer
pixel_y_layer = np.zeros((raster_shape[:]), dtype=np.int32)  # Create pixel_y layer

# Populate coordinate layers
for i in range(raster_shape[0]):
    for j in range(raster_shape[1]):
        pixel_x_layer[i, j] = j  # Assign pixel_x value
        pixel_y_layer[i, j] = i  # Assign pixel_y value

srack them

In [ ]:
from osgeo import gdal
import numpy as np

# List of input raster files to stack
#file_list

# Open the first input raster to get the metadata
first_dataset = gdal.Open(file_list[0], gdal.GA_ReadOnly)
driver = gdal.GetDriverByName('GTiff')

# Create the output dataset for the stacked raster
# add two layer for add x and y coordinates
output_dataset = driver.Create(TifExportUrl + 'stacked_raster.tif', first_dataset.RasterXSize, first_dataset.RasterYSize,
                               len(file_list)+ 2, first_dataset.GetRasterBand(1).DataType)

# Copy projection information from the first dataset
output_dataset.SetProjection(first_dataset.GetProjection())

# Set geotransform
output_dataset.SetGeoTransform(first_dataset.GetGeoTransform())
# Initialize list for stacked arrays
stacked_array_list = []

# Write the stacked array to the output dataset
for i, file  in enumerate(file_list):
    src = gdal.Open(file, gdal.GA_ReadOnly)
    band_array = src.GetRasterBand(1).ReadAsArray()
    output_band = output_dataset.GetRasterBand(i + 1)
    output_band.WriteArray(band_array)
    output_band.FlushCache()
    '''
    # for last layer landUse if 3d
    if src.RasterCount == 3:
      # green band
        output_band = src.GetRasterBand(2).ReadAsArray()
        output_band = output_dataset.GetRasterBand(i + 2)
        output_band.WriteArray(band_array)
        output_band.FlushCache()
      # blue band
        output_band = src.GetRasterBand(3).ReadAsArray()
        output_band = output_dataset.GetRasterBand(i + 3)
        output_band.WriteArray(band_array)
        output_band.FlushCache()
    '''
    src = None
output_band = output_dataset.GetRasterBand(len(file_list)+1)
output_band.WriteArray(pixel_x_layer)
output_band.FlushCache()
output_band = output_dataset.GetRasterBand(len(file_list)+2)
output_band.WriteArray(pixel_y_layer)
output_band.FlushCache()
output_dataset.FlushCache()
output_dataset = None
# Open the first input raster to get the metadata
stackedRaster = gdal.Open(TifExportUrl + 'stacked_raster.tif', gdal.GA_ReadOnly)
print(stackedRaster.ReadAsArray().shape)


(26, 1461, 2351)


In [ ]:
stackedRaster = gdal.Open(TifExportUrl + 'stacked_raster.tif', gdal.GA_ReadOnly)


normalized stack using use the min-max normalization technique.

In [ ]:
import numpy as np
stackArray = stackedRaster.ReadAsArray()
# Assuming your stack has shape (num_layers, height, width)

# Reshape the stack to (num_layers, num_pixels)
reshaped_stack = stackArray.reshape(stackArray.shape[0], -1)

# Compute the minimum and maximum values for each layer
min_vals = reshaped_stack.min(axis=1)
max_vals = reshaped_stack.max(axis=1)

# Perform min-max normalization
normalized_stack = (reshaped_stack - min_vals[:, np.newaxis]) / (max_vals - min_vals)[:, np.newaxis]

# Reshape the normalized stack back to the original shape
normalized_stack = normalized_stack.reshape(stackArray.shape)

<ipython-input-73-b768f73f8a91>:13: RuntimeWarning: invalid value encountered in true_divide
  normalized_stack = (reshaped_stack - min_vals[:, np.newaxis]) / (max_vals - min_vals)[:, np.newaxis]


In [ ]:
stackList = []
for i in range(len(file_list)):
     stackList.append(file_list[i].rsplit('/',1)[-1])
stackList.append('pixel_x')
stackList.append('pixel_y')

In [ ]:
from osgeo import gdal
import numpy as np

# List of input raster files to stack
#file_list

# Open the first input raster to get the metadata
first_dataset = gdal.Open(file_list[0], gdal.GA_ReadOnly)
driver = gdal.GetDriverByName('GTiff')

# Create the output dataset for the stacked raster
# add two layer for add x and y coordinates
output_dataset = driver.Create(TifExportUrl + 'stacked_raster_normalized.tif', first_dataset.RasterXSize, first_dataset.RasterYSize,
                               len(file_list)+ 2, first_dataset.GetRasterBand(1).DataType)

# Copy projection information from the first dataset
output_dataset.SetProjection(first_dataset.GetProjection())

# Set geotransform
output_dataset.SetGeoTransform(first_dataset.GetGeoTransform())
# Initialize list for stacked arraysS
stacked_array_list = []

# Write the stacked array to the output dataset
for i in range(len(stackList)):
    band_array = normalized_stack[i]
    output_band = output_dataset.GetRasterBand(i + 1)
    output_band.WriteArray(band_array)
    output_band.FlushCache()
    src = None
output_dataset.FlushCache()
output_dataset = None
# Open the first input raster to get the metadata
stackedNormalRaster = gdal.Open(TifExportUrl + 'stacked_raster_normalized.tif', gdal.GA_ReadOnly)
print(stackedRaster.ReadAsArray().shape)

(26, 1461, 2351)


In [ ]:
 normalized_stack[26]

(1461, 2351)

In [ ]:
 stackArray[25]

array([[0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [1.000e+00, 1.000e+00, 1.000e+00, ..., 1.000e+00, 1.000e+00,
        1.000e+00],
       [2.000e+00, 2.000e+00, 2.000e+00, ..., 2.000e+00, 2.000e+00,
        2.000e+00],
       ...,
       [1.458e+03, 1.458e+03, 1.458e+03, ..., 1.458e+03, 1.458e+03,
        1.458e+03],
       [1.459e+03, 1.459e+03, 1.459e+03, ..., 1.459e+03, 1.459e+03,
        1.459e+03],
       [1.460e+03, 1.460e+03, 1.460e+03, ..., 1.460e+03, 1.460e+03,
        1.460e+03]])

get values from raters at speciefiec location

In [ ]:
TifExportUrl = '/content/drive/My Drive/Tif_Export/'
ShapefilesURL='/content/drive/My Drive/ShapeFiles/'
mainURL = '/content/drive/My Drive/'
AsthemaricURL = '/content/drive/My Drive/AsthematicData/'
AsthematicSHP = gpd.read_file(AsthemaricURL + 'AsthmaData.shp')
#AsthematicSHP = AsthematicSHP[['UTM_E', 'UTM_N', 'Value', 'geometry']]
#AsthematicSHP.index = range(len(AsthematicSHP))
x_climatology = AsthematicSHP["geometry"].x
y_climatology = AsthematicSHP["geometry"].y

# Create list of XY coordinate pairs
coords = [(x,y) for x, y in zip(AsthematicSHP["geometry"].x, AsthematicSHP["geometry"].x)]


In [ ]:
 # Open the raster and store metadata
src = rasterio.open(TifExportUrl + 'stacked_raster.tif')
transform = src.transform
def GetValuesFromStack(rasterName,raterband):
    # get value from grid
  rasterValues = []
  for i in range(len(x_climatology)):
    value = next(src.sample([(x_climatology[i], y_climatology[i])]))[raterband]
    rasterValues.append(value)

  AsthematicSHP[rasterName] = rasterValues

In [ ]:
TifExportUrl = '/content/drive/My Drive/Tif_Export/'
ShapefilesURL='/content/drive/My Drive/ShapeFiles/'
mainURL = '/content/drive/My Drive/'
for i in range(len(stackList)):
     GetValuesFromStack(stackList[i],i)

Add X and Y pixel to coordinate

In [ ]:
display(AsthematicSHP)

,IsAsthma,geometry,h_Map_Extend.tif,vv_Map_Extend.tif,n_Map_Extend.tif,dd_Map_Extend.tif,ff_Map_Extend.tif,t_Map_Extend.tif,td_Map_Extend.tif,p0_Map_Extend.tif,...,ff_gust_Map_Extend.tif,dd_gust_Map_Extend.tif,newSlop.tif,newElavation.tif,newDistanceToRoad.tif,Tehran_NDVIimage_sentinel20MUTM.tif,newAirpolution.tif,newLandUse.tif,pixel_x,pixel_y
0,0,POINT (522235.379 3946336.774),2307.342371,9018.272095,1.878706,154.460435,2.256086,17.733465,0.541149,873.185826,...,12.097741,191.903886,2.177044,1144.0,0.000000,0.003825,91.773184,255.0,711.0,943.0
1,0,POINT (542851.425 3942391.869),2124.646956,9566.749902,1.878706,154.460435,2.003444,16.509742,0.575663,859.516636,...,12.097741,195.755199,1.470116,1101.0,0.000000,0.068256,73.681585,110.0,1742.0,1140.0
2,0,POINT (518284.980 3954954.567),2452.136866,8671.915979,1.878706,154.460435,1.942819,17.916928,0.395811,868.430838,...,12.097741,190.820441,0.927410,1292.0,0.000000,0.063663,84.146625,143.0,513.0,512.0
3,0,POINT (540470.677 3946202.360),2190.413176,9487.354162,1.878706,154.460435,1.876413,16.580127,0.377256,858.721698,...,12.097741,196.246026,2.177032,1140.0,63.245552,0.004345,75.174944,110.0,1623.0,949.0
4,0,POINT (530717.890 3958792.077),2400.839432,9195.754877,1.878706,154.460435,1.630936,17.048925,-0.007886,859.723456,...,12.097741,195.968340,30.346060,1548.0,196.977158,0.003308,64.092954,110.0,1135.0,320.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1475,1,POINT (538624.136 3937266.788),2128.377386,9508.418440,1.878706,154.460435,2.250299,17.047717,0.824131,869.044032,...,12.097741,195.969959,2.937288,1047.0,335.261078,0.035380,86.071176,85.0,1530.0,1396.0
1476,1,POINT (538731.284 3937407.420),2128.705447,9509.674524,1.878706,154.460435,2.242907,17.034552,0.817243,868.792916,...,12.097741,195.962238,1.469622,1048.0,310.483490,0.029629,86.111093,110.0,1536.0,1389.0
1477,1,POINT (538668.263 3937499.511),2129.961962,9507.730493,1.878706,154.460435,2.241345,17.034865,0.811350,868.764319,...,12.097741,195.962299,6.318957,1050.0,220.000000,0.026989,86.261212,110.0,1532.0,1385.0
1478,1,POINT (541306.414 3937426.592),2109.285622,9558.695062,1.878706,154.460435,2.192135,16.873967,0.841039,866.408164,...,12.097741,195.815681,1.469622,1045.0,63.245552,0.031407,81.991735,121.0,1664.0,1388.0


In [ ]:
AsthematicSHP.describe()

,IsAsthma,h_Map_Extend.tif,vv_Map_Extend.tif,n_Map_Extend.tif,dd_Map_Extend.tif,ff_Map_Extend.tif,t_Map_Extend.tif,td_Map_Extend.tif,p0_Map_Extend.tif,rrr_Map_Extend.tif,...,ff_gust_Map_Extend.tif,dd_gust_Map_Extend.tif,newSlop.tif,newElavation.tif,newDistanceToRoad.tif,Tehran_NDVIimage_sentinel20MUTM.tif,newAirpolution.tif,newLandUse.tif,pixel_x,pixel_y
count,1480.000000,1480.000000,1480.000000,1480.000000,1480.000000,1480.000000,1480.000000,1480.000000,1480.000000,1480.000000,...,1480.000000,1480.000000,1480.000000,1480.000000,1480.000000,1480.000000,1480.000000,1480.000000,1480.000000,1480.000000
mean,0.500000,2248.961923,9325.473886,1.878317,154.460074,1.776522,16.819369,0.347907,859.476986,0.209569,...,12.097723,195.642197,5.276173,1272.482432,107.775518,0.020795,76.585552,127.472973,1366.972297,693.784459
std,0.500169,138.744938,231.080226,0.013755,0.009416,0.293214,0.627682,0.274928,8.601608,0.040488,...,0.000666,1.978999,5.067603,173.150088,101.455825,0.021232,9.653468,43.864202,393.064123,306.550341
min,0.000000,1896.352582,8525.391129,1.350508,154.103593,1.025744,14.327245,-0.356916,829.302795,0.164586,...,12.072114,182.453288,0.000000,1045.000000,0.000000,-0.053986,61.368182,70.000000,60.000000,39.000000
25%,0.000000,2162.586524,9200.510527,1.878706,154.460435,1.588197,16.463650,0.169169,854.190435,0.177928,...,12.097741,195.360191,2.177637,1140.750000,20.000000,0.006801,68.958892,110.000000,1112.000000,466.000000
50%,0.500000,2270.589774,9393.691430,1.878706,154.460435,1.824899,16.875324,0.320339,861.522884,0.196874,...,12.097741,196.091884,3.704792,1210.000000,80.000000,0.017456,74.887782,110.000000,1442.500000,714.500000
75%,1.000000,2359.455100,9494.382364,1.878706,154.460435,2.002470,17.233403,0.481228,866.140664,0.228016,...,12.097741,196.813429,6.507657,1370.000000,170.880081,0.032255,83.891341,110.000000,1633.000000,930.000000
max,1.000000,2495.407367,9723.264459,1.880402,154.460435,2.301996,18.126286,1.193435,873.185826,0.316662,...,12.097741,197.823217,36.478802,1940.000000,400.000000,0.149016,96.832364,255.000000,2303.000000,1430.000000


In [ ]:
AsthematicSHP.to_file(AsthemaricURL + 'AsmethicWithValues.shp')


<ipython-input-71-3de314c8636d>:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  AsthematicSHP.to_file(AsthemaricURL + 'AsmethicWithValues.shp')


save as normal atrribute

In [ ]:

 # Open the raster and store metadata
src = rasterio.open(TifExportUrl + 'stacked_raster_normalized.tif')
transform = src.transform
def GetValuesFromStack(rasterName,raterband):
    # get value from grid
  rasterValues = []
  for i in range(len(x_climatology)):
    value = next(src.sample([(x_climatology[i], y_climatology[i])]))[raterband]
    rasterValues.append(value)

  AsthematicSHP[rasterName] = rasterValues

In [ ]:
TifExportUrl = '/content/drive/My Drive/Tif_Export/'
ShapefilesURL='/content/drive/My Drive/ShapeFiles/'
mainURL = '/content/drive/My Drive/'
for i in range(len(stackList)):
     GetValuesFromStack(stackList[i],i)

In [ ]:
display(AsthematicSHP)


,IsAsthma,geometry,h_Map_Extend.tif,vv_Map_Extend.tif,n_Map_Extend.tif,dd_Map_Extend.tif,ff_Map_Extend.tif,t_Map_Extend.tif,td_Map_Extend.tif,p0_Map_Extend.tif,...,ff_gust_Map_Extend.tif,dd_gust_Map_Extend.tif,newSlop.tif,newElavation.tif,newDistanceToRoad.tif,Tehran_NDVIimage_sentinel20MUTM.tif,newAirpolution.tif,newLandUse.tif,pixel_x,pixel_y
0,0,POINT (522235.379 3946336.774),0.723744,0.368168,0.743858,1.0,0.596653,0.905410,0.458071,0.797379,...,0.66845,0.706565,0.032383,0.067606,0.000000,0.404536,0.852860,1.000000,0.302553,0.645890
1,0,POINT (542851.425 3942391.869),0.461619,0.758035,0.743858,1.0,0.477426,0.629396,0.475423,0.580633,...,0.66845,0.896765,0.021868,0.043380,0.000000,0.515283,0.353632,0.216216,0.741277,0.780822
2,0,POINT (518284.980 3954954.567),0.931490,0.121972,0.743858,1.0,0.448816,0.946791,0.384998,0.721981,...,0.66845,0.653059,0.013795,0.150986,0.000000,0.507389,0.642409,0.394595,0.218298,0.350685
3,0,POINT (540470.677 3946202.360),0.555978,0.701599,0.743858,1.0,0.417477,0.645272,0.375669,0.568028,...,0.66845,0.921005,0.032383,0.065352,0.158114,0.405428,0.394840,0.216216,0.690638,0.650000
4,0,POINT (530717.890 3958792.077),0.857890,0.494326,0.743858,1.0,0.301632,0.751010,0.182028,0.583912,...,0.66845,0.907291,0.451392,0.295211,0.492443,0.403647,0.089038,0.216216,0.482979,0.219178
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1475,1,POINT (538624.136 3937266.788),0.466972,0.716572,0.743858,1.0,0.593921,0.750738,0.600348,0.731704,...,0.66845,0.907371,0.043692,0.012958,0.838153,0.458775,0.695516,0.081081,0.651064,0.956164
1476,1,POINT (538731.284 3937407.420),0.467442,0.717465,0.743858,1.0,0.590433,0.747768,0.596885,0.727723,...,0.66845,0.906990,0.021860,0.013521,0.776209,0.448888,0.696618,0.216216,0.653617,0.951370
1477,1,POINT (538668.263 3937499.511),0.469245,0.716083,0.743858,1.0,0.589696,0.747839,0.593922,0.727269,...,0.66845,0.906993,0.093993,0.014648,0.550000,0.444352,0.700760,0.216216,0.651915,0.948630
1478,1,POINT (541306.414 3937426.592),0.439579,0.752310,0.743858,1.0,0.566473,0.711548,0.608849,0.689909,...,0.66845,0.899752,0.021860,0.011831,0.158114,0.451945,0.582946,0.275676,0.708085,0.950685


In [ ]:
AsthematicSHP.describe()


,IsAsthma,h_Map_Extend.tif,vv_Map_Extend.tif,n_Map_Extend.tif,dd_Map_Extend.tif,ff_Map_Extend.tif,t_Map_Extend.tif,td_Map_Extend.tif,p0_Map_Extend.tif,rrr_Map_Extend.tif,...,ff_gust_Map_Extend.tif,dd_gust_Map_Extend.tif,newSlop.tif,newElavation.tif,newDistanceToRoad.tif,Tehran_NDVIimage_sentinel20MUTM.tif,newAirpolution.tif,newLandUse.tif,pixel_x,pixel_y
count,1480.000000,1480.000000,1480.000000,1480.000000,1480.000000,1480.000000,1480.000000,1480.000000,1480.000000,1480.000000,...,1480.000000,1480.000000,1480.000000,1480.000000,1480.000000,1480.000000,1480.000000,1480.000000,1480.000000,1480.000000
mean,0.500000,0.639982,0.586532,0.743682,0.999922,0.370337,0.699233,0.360913,0.580004,0.285387,...,0.668444,0.891184,0.078482,0.139990,0.269439,0.433704,0.433765,0.310665,0.581690,0.475195
std,0.500169,0.199066,0.164256,0.006254,0.002023,0.138373,0.141575,0.138227,0.136392,0.254873,...,0.000240,0.097734,0.075380,0.097549,0.253640,0.036495,0.266383,0.237104,0.167261,0.209966
min,0.000000,0.134071,0.017820,0.503681,0.923320,0.016030,0.137128,0.006544,0.101546,0.002216,...,0.659209,0.239841,0.000000,0.011831,0.000000,0.305167,0.013850,0.000000,0.025532,0.026712
25%,0.000000,0.516054,0.497706,0.743858,1.000000,0.281462,0.619000,0.271047,0.496178,0.086206,...,0.668450,0.877257,0.032392,0.065775,0.050000,0.409651,0.223311,0.216216,0.473191,0.319178
50%,0.500000,0.671013,0.635022,0.743858,1.000000,0.393167,0.711854,0.347052,0.612445,0.205474,...,0.668450,0.913392,0.055108,0.104789,0.200000,0.427965,0.386916,0.216216,0.613830,0.489384
75%,1.000000,0.798513,0.706595,0.743858,1.000000,0.476966,0.792620,0.427944,0.685667,0.401512,...,0.668450,0.949026,0.096800,0.194930,0.427200,0.453403,0.635365,0.216216,0.694894,0.636986
max,1.000000,0.993572,0.869288,0.744629,1.000000,0.618319,0.994012,0.786026,0.797379,0.959542,...,0.668450,0.998896,0.542616,0.516056,1.000000,0.654098,0.992466,1.000000,0.980000,0.979452


In [ ]:
AsthematicSHP.to_file(AsthemaricURL + 'AsmethicWithValues_normalized.shp')


Get value at air pollution location

In [ ]:
TifExportUrl = '/content/drive/My Drive/Tif_Export/'
ShapefilesURL='/content/drive/My Drive/ShapeFiles/'
mainURL = '/content/drive/My Drive/'
AsthemaricURL = '/content/drive/My Drive/AsthematicData/'
AsthematicSHP = gpd.read_file(AsthemaricURL + 'AsthmaData.shp')
#AsthematicSHP = AsthematicSHP[['UTM_E', 'UTM_N', 'Value', 'geometry']]
#AsthematicSHP.index = range(len(AsthematicSHP))
x_climatology = AsthematicSHP["geometry"].x
y_climatology = AsthematicSHP["geometry"].y

# Create list of XY coordinate pairs
coords = [(x,y) for x, y in zip(AsthematicSHP["geometry"].x, AsthematicSHP["geometry"].x)]

In [ ]:

 # Open the raster and store metadata
src = rasterio.open(TifExportUrl + 'stacked_raster_normalized.tif')
transform = src.transform
def GetValuesFromStack(rasterName,raterband):
    # get value from grid
  rasterValues = []
  for i in range(len(x_climatology)):
    value = next(src.sample([(x_climatology[i], y_climatology[i])]))[raterband]
    rasterValues.append(value)

  AsthematicSHP[rasterName] = rasterValues

In [ ]:
TifExportUrl = '/content/drive/My Drive/Tif_Export/'
ShapefilesURL='/content/drive/My Drive/ShapeFiles/'
mainURL = '/content/drive/My Drive/'
for i in range(len(stackList)):
     GetValuesFromStack(stackList[i],i)